In [ ]:
import numpy as np
import pandas as pd

#pd.set_option('display.max_rows', 500)

In [ ]:
# Abre os arquivos
treino = pd.read_csv('train.csv')
teste = pd.read_csv('test.csv')

# Damos uma espiada no data frame
treino.head()

Sempre legal dar uma olhada nas estatísticas do data frame. Aqui já temos noção de como os dados estão, por exemplo se tem Age negativos, ou a Age com mais de 200 anos.. ja pensou?

In [ ]:
treino.describe()

Depois que a gente conheceu os dados de treino, vamos ver sem tem features com valures NULL

In [ ]:
treino.isnull().sum().sort_values()

In [ ]:
teste.isnull().sum().sort_values()

Idade nula é um porblema. Vamos resolver isso assim: Subsituir os nulos pela média, mas vamos fazer de forma aleatória levando em conta o range dentro do desvio padrão.

In [ ]:
for df_to_work in [teste, treino]:
    media = df_to_work['Age'].mean()
    desvio = df_to_work['Age'].std()
    total_nulos = df_to_work['Age'].isnull().sum()

    # Cria um array com a quantidade de Ages em branco
    total_nulos_randomicos = np.random.random_integers(media-desvio, media+desvio,size=total_nulos)
    # Aplica o array nos registros do data frame que estao com idade null
    df_to_work['Age'][np.isnan(df_to_work['Age'])] = total_nulos_randomicos



In [ ]:
# Vamos ver se deu bom...
print('Age null em treino: ', treino['Age'].isnull().sum())
print('Age null em teste: ', teste['Age'].isnull().sum())

O campo sexo precisa ser transformado em binário.

In [ ]:
# Tratamentos
for df_to_work in [teste, treino]:
    df_to_work['Sex_binario'] = df_to_work['Sex'].apply(lambda x: 1 if x == 'female' else 0)

Vamos jogar os caras null para -1

In [ ]:
treino.fillna(value=-1)
teste.fillna(value=-1)

Vamos criar umas features... pensar em algumas hipóteses.

TREINO

In [ ]:
for df_to_work in [teste, treino]:

    # Se reparar nos nomes, existem alguns títulos.
    df_to_work['is_mr'] = df_to_work['Name'].str.contains('Mr.')
    df_to_work['is_mrs'] = df_to_work['Name'].str.contains('Mrs.')
    df_to_work['is_miss'] = df_to_work['Name'].str.contains('Miss.')
    df_to_work['is_rev'] = df_to_work['Name'].str.contains('Rev.')
    df_to_work['is_master'] = df_to_work['Name'].str.contains('Master.')
    df_to_work['is_dr'] = df_to_work['Name'].str.contains('Dr.')
    df_to_work['is_misc'] = df_to_work['Name'].str.contains('Misc.')

    # Junta a informação se o passageiro esta sozinho, com filhos e/ou marido. Seria o total de pessoas da familia viajando juntos, algo assim.
    df_to_work['family_size'] = df_to_work.apply(lambda row: int(row['SibSp'] + row['Parch'])>0, axis=1)+1

    df_to_work['is_alone'] = 1
    df_to_work['is_alone'] = df_to_work.apply(lambda row: row['family_size']>1, axis=1)

    # Uma classificação por faixa etária
    df_to_work['faixa_idade'] = df_to_work['Age']/12
    # Se é criança
    df_to_work['is_children'] = df_to_work.apply(lambda row: row['Age']<15, axis=1)
    # Se é idoso
    df_to_work['is_old'] = df_to_work.apply(lambda row: row['Age']>60, axis=1)

    # Prmeira letra da cabine, imaginei que isso pode influenciar pelo fato de estar no inicio/final de corredor, andar diferente, enfim... 
    df_to_work['cabin_fst_letter'] = df_to_work['Cabin'].str[0:1]
    df_to_work['cabin_fst_letter'].replace(np.nan, -1, inplace=True) # soh acerta o que ficou como NULL

    df_to_work['cabine_is_null'] = df_to_work['Cabin'].isnull().astype(int)

    df_to_work['embarked_s'] = (df_to_work['Embarked'] == 'S').astype(int)
    df_to_work['embarked_c'] = (df_to_work['Embarked'] == 'C').astype(int)

    df_to_work['is_rich'] = (df_to_work['Pclass'] < 2).astype(int)



In [ ]:
df_to_work.head()

Muito importante dar um check na correlação das variáveis com o nosso target. Vamos usar isso para limpar o data frame ou para excluir alguma feature na hora da predição.

In [ ]:
#treino.corr().abs()[["Survived"]]

In [ ]:
# Pinto os nomes das colunas pra conseguir copiar e colar abaixo :)
treino.columns.values

In [ ]:
# Vou usar essas colunas como preditoras
colunas = ['Sex_binario', 'Pclass',  'Embarked', 'Fare', 'SibSp', 'Parch',
           'is_mr', 'is_mrs', 'is_miss', 'is_rev', 'is_master', 'is_misc',
            'is_dr', 'family_size', 'is_alone', 'cabin_fst_letter', 'faixa_idade',
            'is_children', 'is_old', 'cabine_is_null', 'embarked_s', 'embarked_c'
]

In [ ]:
# Transformo algumas features categoricas
for coluna in ['Cabin', 'Embarked', 'cabin_fst_letter']:
    treino[coluna]=pd.Categorical(treino[coluna]).codes
    teste[coluna]=pd.Categorical(teste[coluna]).codes

Agora sim, vamos pa predição

In [ ]:
entradas = colunas
saida = 'Survived'

X = treino[entradas]
y = treino[saida]
X = X.fillna(-1)

from sklearn.model_selection import train_test_split


In [ ]:
# validacao cruzada
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
print('Colunas:', X.columns.values)
resultados = []
kf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=10)
#modelo = RandomForestClassifier(n_estimators=1000, n_jobs=-1,random_state=1, max_depth=50)
modelo = LogisticRegression()
for linhas_treino, linhas_teste in kf.split(X):

    X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_teste]
    y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_teste]
    
    modelo.fit(X_treino, y_treino)

    p = modelo.predict(X_valid)

    acc = np.mean(y_valid == p)
    print('Acuracia: ', acc)
    resultados.append(acc)
        
print('Media acuracia:',  np.mean(resultados))


Com misc: 0.8226643825263263
family size and is_alone  0.8223310827832921
is_rick = 0.8214319544515544



In [ ]:
predict = modelo.predict(teste[entradas].fillna(-1))

# para facilitar o nome do arquivo
file_saida = input('Digite um nome de arquivo para gerar uma submissao: ')

if file_saida != '':
    sub = pd.Series(predict, index=teste['PassengerId'], name='Survived')
    sub.to_csv(file_saida + '.csv')
    print('Arquivo gerado com sucesso :)')